In [22]:
import os


# 현재 작업 디렉토리의 절대 경로를 얻는다
ROOT_DIR = os.path.abspath(os.curdir)

In [2]:
#데이터 불러오기 (train 데이터의 10% 비율로 test 데이터 생성)

In [23]:
import os
import glob
import shutil


# 비율 설정 10 % 
ratio = 0.1

# 디렉토리 경로를 결합  
src_root_dir = os.path.join(ROOT_DIR,'./dataset/cats_n_dogs_emotion/train/')
dst_root_dir = os.path.join(ROOT_DIR,'./dataset/cats_n_dogs_emotion/test/')

# src_root_dir 에 있는 파일과 디렉토리의 이름으로 이룩어진 리스트 반환 
label_name_list = os.listdir(src_root_dir)


# 반환된 리스트에서 요소를 하나씩 뽑아 dst_root_dir 경로와 결합 
for label_name in label_name_list:
    dst_label_name_dir = dst_root_dir + label_name

# 해당 디렉토리가 존재하지 않는다면 해당경로에 새로운 디렉토리 생성  
    if not os.path.exists(dst_label_name_dir):
        os.mkdir(dst_label_name_dir)
        

# 해당 경로에 매칭되는 파일 목록 가져오기 
# ratio 비율과 첫번째 변수의 길이를 곱하여 정수로 변환 
# split_num 으로 요소 분해 하여 학습이미지중 일부를 테스트 이미지로 분리
for label_name in label_name_list:
    train_image_file_list = glob.glob(src_root_dir+label_name+'/*')
    split_num = int(ratio*len(train_image_file_list))
    test_image_file_list = train_image_file_list[0:split_num]

# 테스트 이미지로 분리한 것들을 새로운 디렉토리로 move 
    for image_file in test_image_file_list:
        shutil.move(image_file, dst_root_dir+label_name)

In [4]:
# 데이터 불러오기 (test 디렉토리 파일 -> test_image_files 디렉토리로 복사)

In [24]:
import os 
import glob 
import shutil

# 디렉토리 경로를 결합  
src_root_dir = os.path.join(ROOT_DIR,'./dataset/cats_n_dogs_emotion/test/')
dst_root_dir = os.path.join(ROOT_DIR,'./dataset/cats_n_dogs_emotion/test_image_files/')

# 반환된 리스트에서 요소를 하나씩 뽑아 dst_root_dir 경로와 결합 
label_name_list = os.listdir(src_root_dir)

# 해당 경로에 매칭되는 파일 목록 가져오기
# 해당 레이블의 이미지 파일수 출력 
for label_name in label_name_list:
    image_file_list = glob.glob(src_root_dir+label_name+'/*')
    print('total [%s] image file nums => [%s]' % (label_name, len(image_file_list)))
    
    # 변수 초기화 
    copy_nums = 0 
    
    #imgae_file 을 dst_root_dir 로 복사 
    # 복사된 이미지 파일 갯수 출력
    for image_file in image_file_list:
        shutil.copy(image_file, dst_root_dir)
        copy_nums = copy_nums + 1 
        
    print('total copy nums =>', copy_nums)  

total [angry] image file nums => [149]
total copy nums => 149
total [happy] image file nums => [67]
total copy nums => 67
total [sad] image file nums => [99]
total copy nums => 99


In [6]:
# ImageDataGenerator 정의 

In [25]:
# 이미지 크기 변수에 저장
IMG_WIDTH = 224 
IMG_HEIGHT = 224 

# 디렉토리 경로를 결합  
train_dir = os.path.join(ROOT_DIR, './dataset/cats_n_dogs_emotion/train/')
validation_dir = os.path.join(ROOT_DIR, './dataset/cats_n_dogs_emotion/train/')
test_dir = os.path.join(ROOT_DIR,'./dataset/cats_n_dogs_emotion/test/') 

In [26]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# train 데이터로 부터 15% 비율로 validation data 생성
# 이미지 데이터 전처리
# rescale 이미지 데이터의 픽셀 값을 0과 1사이의 값으로 재조정
# 신경망 모델의 학습을 더 잘 수행 할 수 있도록
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                                  height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                  validation_split=0.15)
# 이미지 데이터 전치리 및 데이터 생성에 사용  
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.15)



In [27]:

# train_dir  : 학습 데이터가 있는 디렉토리 경로지정 
# batch_size : 생성할 배치 사이즈 16
# color_mode : 이미지 컬러 모드를 지정
# class_mode : 클래스 모드 지정 sparse : 정수형 레이블 반환 
# subset     : 이터의 하위 집합 지정 
#target_size : 이미지의 타겟 크기를 지정
# flow_from_directory : 데이터 생성기를 생성하고, 이미지를 로드하여 배치 단위로 반환
train_generator = train_datagen.flow_from_directory(train_dir, batch_size=16, color_mode='rgb',
                                                   class_mode='sparse', subset='training',
                                                   target_size=(IMG_WIDTH,IMG_HEIGHT))


validation_generator = validation_datagen.flow_from_directory(validation_dir, batch_size=16, color_mode='rgb', 
                                                             class_mode='sparse', subset= 'validation',
                                                             target_size=(IMG_WIDTH,IMG_HEIGHT))
 

Found 397 images belonging to 3 classes.
Found 67 images belonging to 3 classes.


In [28]:
#해당 클래스 인덱스 확인
print(train_generator.class_indices)

{'angry': 0, 'happy': 1, 'sad': 2}


In [11]:
# 모델구축 (Pre-Trained MobileNet + User-Defined Classfier)

In [29]:
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input, decode_predictions


# MobileNet은 사전 훈련된 가중치를 사용하여 이미지 분류 및 특성 추출과 같은 작업에 사용되는 네트워크 아키텍처입니다.
# weights : 사전 훈련된 ImageNet 데이터셋의 가중치를 사용하도록 설정하는 매개변수 
# include_top=False : 네트워크의 상단에 있는 완전 연결 레이어를 포함하지 않도록 설정하는 매개변수입니다.
# input_shape : 입력 이미지의 형태를 지정하는 매개변수, 이미지 채널수 3 
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

In [30]:
from keras.models import Sequential 
from keras.layers import Dense, Activation, Flatten, Dropout
#Sequential 모델 생성 
model = Sequential()
 
# MoblileNet 모델을 추가 
model.add(base_model)

# 2D 특성맵을 -> 1D 벡터로 변환 
model.add(Flatten())

# user-defined classifier
# 사용자 정의 분류기 32개의 뉴런을 가진 완전 연결 레이어를 추가 
# 활성화 함수로는 relu
model.add(Dense(32, activation='relu'))
# 25% 드롭아웃 적용하는 레이어 추가 과적합을 방지하기위해 임의로 일부 뉴런을 비활성화 
model.add(Dropout(0.25))
# 3개의 클래스에 대한 출력을 생성하는 완전연결 레이어 
# 활성화 함수로는 softmax 
model.add(Dense(3, activation='softmax')) 

In [31]:
import tensorflow as tf 
from keras import optimizers

# loss = 정수형 레이블을 인코딩 된 형태로 처리
# Adam = 경사 하강법의 한 종류, 모델의 가중치 업데이트 
# 2e-5 = 학습률 (엡데이트의 크기를 조절하는 매개변수) 2 * 10^(-5) => 0.00002 
model.compile(loss='sparse_categorical_crossentropy',
             optimizer = tf.keras.optimizers.Adam(2e-5), metrics =['accuracy'])

# 모델 구조 요약 출력 
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 50176)             0         
                                                                 
 dense_2 (Dense)             (None, 32)                1605664   
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 3)                 99        
                                                                 
Total params: 4,834,627
Trainable params: 4,812,739
Non-trainable params: 21,888
_______________________________________

In [15]:
# 모델에 학습 

In [1]:
from tensorflow.keras.callbacks import EarlyStopping

# 조기종료 구현 
# EarlyStopping:  지정된 기준에 따라 모델의 학습을 조기에 종료할 수 있는 콜백 함수 
# patience = 5 :  조기종료 판단 기준, 검증 손실이 5번의 에포크 동안 개선되지 않으면 학습중지, 최상의 모델 저장 
earlystopping = EarlyStopping(monitor = 'val_loss', patience=5)

# fit 함수로 학습 진행 
# 학습데이터, 검증데이터, 에포크 횟수, 조기종료 구현 함수 등을 포함
hist=model.fit(train_generator, validation_data=validation_generator, epochs= 20, callbacks=[earlystopping])

NameError: name 'model' is not defined

In [33]:
# 학습된 모델을 계속적으로 학습시킬수 없으니 HDF5 파일 형식의 Keras 모델을 지정한 경로에 저장 
model.save('./savedata/emotion.h5')
# 저장된 학습모델 불러오기 
#h5_model=tf.keras.models.load_model('./savedata/emotion.h5')

In [2]:
import random
import os 
import numpy as np 
import cv2 
import glob 

# 해당 레이블에 대응하는 정수 인데스를 매핑하는 딕셔너리를 생성
label_dict = {'angry':0, 'happy':1, 'sad':2}

# 지정된 경로에 있는 모든 .jpg 확장자 파일들의 목록을 가져옴 
test_image_files_list = glob.glob(ROOT_DIR + './dataset/cats_n_dogs_emotion/test_image_files/*.jpg')

# 무작위로 파일들을 섞음
random.shuffle(test_image_files_list)

# 16개의 파일 테스트할 예정
test_num = 16
test_image_files = test_image_files_list[:test_num] # 테스트 파일 이름은 정답. 숫자. jpg


label_list = []

# 이미지 파일의 경로에서 레이블 추출하여 변수에 담을것, 인덱스를 매핑하기 위해서 
for i in range(len(test_image_files)):
    print(test_image_files[i])
    label = test_image_files[i].split('/')[-1].split('.')[0].strip()
    label2 = label.split('\\')[1]
    print(label1)            
    #label = 'happy'
    label_list.append(label_dict[label2])
    
src_image_list = []




for i in range(len(test_image_files)):
    # imread : 경로에 있는 테스트 이미지 파일을 컬러로 읽어옴 
    src_img = cv2.imread(test_image_files[i], cv2.IMREAD_COLOR)
    
    # resize : 이이미지의 크기 조절 
    src_img = cv2.resize(src_img, dsize=(IMG_WIDTH, IMG_HEIGHT))
    
    # cvtColor : BGR 채널 순서를 RGB 채널 순서로 변경 
    src_img = cv2.cvtColor(src_img, cv2.COLOR_BGR2RGB)
    
    # 이미지를 0과 1사이의 값으로 정규화 
    src_img = src_img / 255.0

    # 전처리된 이미지를 list 에 추가 
    src_image_list.append(src_img)
    


#4차원 텐서 변환 
#배열로 변환함으로써 데이터를 효율적으로 다루고 다양한 배열 연산을 수행할 수 있음
src_image_array = np.array(src_image_list)
label_array = np.array(label_list) 


NameError: name 'ROOT_DIR' is not defined

In [ ]:
# 16 개 테스트 이미지 파일에 대한 prediction 실행 

In [ ]:
# 저장된 모델로 테스트 이미지 예측 
pred = h5_model.predict(src_image_list)
print(pred.shape)

In [ ]:
# 예측 결과와 해당이미지를 시각화 하기위해 
import matplotlib.pyplot as plt 


class_names =['happy', 'sad', 'angry']

# 출력할 그림의 크기를 설정 
plt.figure(figsize=(12,12))

# 예측 결과와 이미지를 하나씩 시각화
for pos in range(len(pred)):
    
    # 서브 플롯이 생성될 행, 열의 갯수 나타냄, 1부터 시작하여 행우선으로 증가
    plt.subplot(4,4,pos+1)
    # 시각화 결과에서 축과 눈금 숨김
    plt.axis('off')
    
    # 정수형 레이블을 해당 문자열로 변환 
    label_str = class_names[label_array[pos]]
    # 예측 결과의 가장 높은값을 인덱스로 가져온후 해당 문자열로 변환 
    pred_str = class_names[np.argmax(pred[pos])]
    
    # 제목 설정 
    plt.title('label:' + label_str + '\npred:' + pred_str)
    
    # 시각화 
    plt.imshow(src_image_array[pos])
# 배치     
plt.tight_layout()
# 출력
plt.show()
